In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import re

# ----------------------------
# 1. Load Models
# ----------------------------
llm_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
llm_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# ----------------------------
# 2. Build Knowledge Base (for RAG tool)
# ----------------------------
knowledge_chunks = [
    "Python is a high-level programming language that emphasizes simplicity and readability.",
    "Java is a versatile, object-oriented programming language designed for portability across platforms.",
    "Python is dynamically typed and concise, while Java is statically typed and verbose."
]

chunk_embeddings = embedder.encode(knowledge_chunks, convert_to_numpy=True)
d = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(chunk_embeddings)

# ----------------------------
# 3. Tools
# ----------------------------
def rag_tool(query, k=2):
    """Retrieve knowledge from FAISS index."""
    query_emb = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_emb, k)
    retrieved = [knowledge_chunks[i] for i in I[0] if i < len(knowledge_chunks)]
    return " ".join(retrieved)

In [2]:

def calculator_tool(query):
    """Evaluate simple math expression from user query."""
    expr = re.findall(r"[\d+\-*/().]+", query)
    if expr:
        try:
            result = eval("".join(expr))
            return f"The result is {result}."
        except:
            return "Sorry, I couldn’t calculate that."
    return "No math expression found."

# ----------------------------
# 4. Agent Controller
# ----------------------------
def agent(query):
    # Step 1: Ask LLM which tool to use
    instruction = f"""
You are a controller agent.
Decide which tool is best for the question.
Available tools:
- RAG: for knowledge questions (Python, Java, etc).
- CALC: for math/calculation questions.

Question: {query}
Answer ONLY 'RAG' or 'CALC'."""

    inputs = llm_tokenizer(instruction, return_tensors="pt", truncation=True)
    outputs = llm_model.generate(**inputs, max_length=10)
    decision = llm_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Step 2: Route to tool
    if "CALC" in decision.upper():
        tool_result = calculator_tool(query)
    else:
        context = rag_tool(query)
        tool_result = context

    return tool_result

# ----------------------------
# 5. Demo
# ----------------------------
print("User: What is Python?")
print("Agent:", agent("What is Python?"))

print("\nUser: What is Java?")
print("Agent:", agent("What is Java?"))

print("\nUser: Compare Python and Java.")
print("Agent:", agent("Compare Python and Java."))

print("\nUser: What is 2 + 5 * 3?")
print("Agent:", agent("What is 2 + 5 * 3?"))


User: What is Python?
Agent: Python is a high-level programming language that emphasizes simplicity and readability. Python is dynamically typed and concise, while Java is statically typed and verbose.

User: What is Java?
Agent: Java is a versatile, object-oriented programming language designed for portability across platforms. Python is dynamically typed and concise, while Java is statically typed and verbose.

User: Compare Python and Java.
Agent: Python is dynamically typed and concise, while Java is statically typed and verbose. Python is a high-level programming language that emphasizes simplicity and readability.

User: What is 2 + 5 * 3?
Agent: The result is 17.
